In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-524e5a73-eff4-81a0-9d0d-cdc5b88972ae)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-06-27 12:33:25--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-27 12:33:25 (105 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-06-27 12:33:25--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.45.112, 142.250.81.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-06-27 12:33:25 (163 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import helper_functions as hf
import numpy as np
import pandas as pd

In [ ]:
hf.unzip_data('nlp_getting_started.zip')

In [ ]:
train_data = pd.read_csv('train.csv')

In [ ]:
train_data.shape

(7613, 5)

In [ ]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
train_data_shuffled = train_data.sample(frac =1, random_state=42)
train_data_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_data_shuffled[['text','target']]

,text,target
2644,So you have a new weapon that can cause un-ima...,1
2227,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,Aftershock back to school kick off was great. ...,0
6845,in response to trauma Children of Addicts deve...,0
...,...,...
5226,@Eganator2000 There aren't many Obliteration s...,0
5390,just had a panic attack bc I don't have enough...,0
860,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,Officials say a quarantine is in place at an A...,1


In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                            train_data_shuffled['target'].to_numpy(),
                                                                            test_size = 0.2,
                                                                            random_state = 42)

In [ ]:
type(train_sentences)

numpy.ndarray

In [ ]:
train_sentences[:10]

array(['Detonation fashionable mountaineering electronic watch water-resistant couples leisure tab\x89Û_ http://t.co/E61x9Y65QD http://t.co/OVLET0gDqm',
       "Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/tFan6qq2Ys http://t.co/rAkwWritPo",
       'I-77 Mile Marker 31 to 40 South Mooresville  Iredell Vehicle Accident Congestion at 8/6 1:18 PM',
       "#Greece's tax revenues collapse as debt crisis continues via @guardian #bailout http://t.co/cJvbQXw83s ^mp",
       'Be not afraid of sudden fear neither of the desolation of the wicked when it cometh. For the Lord shall be thy... http://t.co/bP597YDs2b',
       'Storm batters Auckland and Northland: A violent overnight storm has battered Auckland and Northland uprooting... http://t.co/enrPGRgtTs',
       "Benzema increasingly looks to be a casualty of Benitez's new look squad. Arsenal bound? 50-50 chance I think",
       'Acquire your postexistence straight a elevation in addition to uphe

### Text Vectorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None, #no grouping of tokens
                                    output_mode = 'int',
                                    output_sequence_length = None
                                    )

In [ ]:
max_vocab_length = 10000
max_length = 15

In [ ]:
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams=None,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [ ]:
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
sample_sentence = "There's a flood in my street!"

In [ ]:
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[240,   3, 229,   4,  13, 724,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
import random

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence}  \
      \n\n Vectorized text : \n {text_vectorizer([random_sentence])}")

Original text : 
 who said this? Yosemite Sam or Drumpf? ÛÏNobody Û÷ill vote for a flattened out rabbit skin a-huh huh Û_ I always sayÛ        

 Vectorized text : 
 [[  63  209   19 6035 8155   53    1 5942 5915 1229   10    3  477   36
  8677]]


In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['minded', 'mindblowing', 'milne', 'milledgeville', 'millcityio'])

### Embedding layer

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                      embeddings_initializer = 'uniform',
                                      input_length=max_length)

embedding

In [ ]:
embedding(text_vectorizer([random_sentence]))

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02570195,  0.00753912, -0.03872136, ..., -0.02983173,
          0.03114343, -0.04511238],
        [-0.02407273, -0.04724798, -0.02808951, ...,  0.03867514,
          0.01157918, -0.02125242],
        [ 0.03864339,  0.03847064, -0.00150796, ..., -0.01596037,
          0.01954986,  0.02514965],
        ...,
        [-0.00754011, -0.00701193, -0.04383693, ..., -0.00522036,
          0.0348869 ,  0.03561899],
        [ 0.00925856, -0.04578033,  0.0132272 , ...,  0.03983433,
         -0.04181287, -0.01275759],
        [-0.02570318,  0.03081211, -0.049244  , ...,  0.0027961 ,
         -0.04791925, -0.00254972]]], dtype=float32)>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

### Model 0 : Naive Bayes model

In [ ]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

Evaluate

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)

In [ ]:
baseline_score

0.799080761654629

In [ ]:
y_pred = model_0.predict(val_sentences)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')

  model_results = {'accuracy' : model_accuracy,
                   'precision' : model_precision,
                   'recall' : model_recall,
                   'f1' : model_f1}

  return model_results

In [ ]:
calculate_results(val_labels, y_pred)

{'accuracy': 0.799080761654629,
 'f1': 0.7920155324845473,
 'precision': 0.8146358812834972,
 'recall': 0.799080761654629}

### Model 1 : A simple dense model

In [ ]:
import tensorflow as tf
from helper_functions import create_tensorboard_callback

save_dir = 'model_logs'

In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)


model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


history_1 = model_1.fit(x = train_sentences, y = train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 7s 10ms/step - loss: 0.6203 - accuracy: 0.6834 - val_loss: 0.5492 - val_accuracy: 0.7485
Epoch 2/5
191/191 [==============================] - 2s 9ms/step - loss: 0.4551 - accuracy: 0.8131 - val_loss: 0.4792 - val_accuracy: 0.7840
Epoch 3/5
191/191 [==============================] - 2s 9ms/step - loss: 0.3544 - accuracy: 0.8606 - val_loss: 0.4634 - val_accuracy: 0.7879
Epoch 4/5
191/191 [==============================] - 2s 9ms/step - loss: 0.2870 - accuracy: 0.8882 - val_loss: 0.4641 - val_accuracy: 0.7938
Epoch 5/5
191/191 [==============================] - 2s 8ms/step - loss: 0.2379 - accuracy: 0.9117 - val_loss: 0.4743 - val_accuracy: 0.7873


In [ ]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(1523, 1)

In [ ]:
model_1_pred_probs[0]

array([0.44197246], dtype=float32)

In [ ]:
model_1_pred = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [ ]:
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_pred)

In [ ]:
model_1_results

{'accuracy': 0.7872619829284307,
 'f1': 0.7846223664576338,
 'precision': 0.7885084255168797,
 'recall': 0.7872619829284307}

In [ ]:
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights.shape

(10000, 128)

In [ ]:
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# # Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
   if num == 0: 
      continue # skip padding token
   vec = embed_weights[num]
   out_m.write(word + "\n") # write words to file
   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

# # Download files locally to upload to Embedding Projector
try:
   from google.colab import files
except ImportError:
   pass
else:
   files.download("embedding_vectors.tsv")
   files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recurrent Neural Network



### Model 3 : LSTM - Long short term memory

In [ ]:
inputs = tf.keras.layers.Input(shape = (1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
#x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x)
#x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

model_2.summary()


model_2.compile(loss='binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


history_2 = model_2.fit(x = train_sentences,
                        y = train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name=save_dir,
                                                                 experiment_name = 'model_2_LSTM')])

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs

array([[0.24180204],
       [0.59485507],
       [0.99979764],
       ...,
       [0.9982736 ],
       [0.9999027 ],
       [0.9997099 ]], dtype=float32)

In [ ]:
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_pred)

model_2_results

{'accuracy': 0.7734734077478661,
 'f1': 0.7716540576990034,
 'precision': 0.773099971715809,
 'recall': 0.7734734077478661}

### Model 4 GRU - Gated Recurrent unit

In [ ]:
from tensorflow.python.util.dispatch import register_unary_elementwise_api
inputs = tf.keras.layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)

x = tf.keras.layers.GRU(64, return_sequences= True)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs, outputs)


model_4.summary()


model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


history_4 = model_4.fit(train_sentences, train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = save_dir,
                                                                  experiment_name = 'model_4_GRU')])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_6 (GRU)                 (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_2   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_14 (Dense)            (None, 1)                 65  

In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs

array([[0.30792364],
       [0.84461004],
       [0.9996891 ],
       ...,
       [0.98573446],
       [0.99999785],
       [0.9999962 ]], dtype=float32)

In [ ]:
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_pred)

In [ ]:
model_4_results

{'accuracy': 0.7688772160210111,
 'f1': 0.7673217237793988,
 'precision': 0.7682105800686845,
 'recall': 0.7688772160210111}

### Model 5 Bidirectional RNN